In [103]:
import math as mt
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
import matplotlib.patches as patches

class Jeu:
    """ Jeu de blackjack qui prend un croupier et la liste des joueur de la table en parametre. le tas correspond au carte
    pas encore tiré sous la forme d'un tableau de taille 10 qui correspond a la valeur de chaque carte, le sabot le nombres de carte restante dans le sabot
        run(self,draw,pause): permet de joueur une partie, "avec ou sans affichage, avec une pause entre chaque coup."
                Rend le joueur qui a gagne 
        reinit_sabot() permet de changer le sabot.la fonction est utilisé dans run.
        tireCarte() permet de sortir une carte resante dans le sabot aléatoirement de maniere uniformement pondéré suivant les carte restantes
        et met a jour le sabot et le tas
        
    """
    def __init__(self):
        self.tas=[0,24,24,24,24,24,24,24,24,24,96]
        self.sabot=312
    def run(self,ListeJoueur=None,croupier=None):
        print("NOUVELLE PARTIE:")
        ListeJeu=np.zeros((len(ListeJoueur)))    #conserve le jeu du joueur i
        ListeMise=np.zeros((len(ListeJoueur)))  #conserve la mise du joueur i
        ListeAs=np.zeros((len(ListeJoueur)))    #indique la precense d'un as ou plusieurs dans le jeu du joueur
        """les joueurs mises"""
        print("les joueurs mises:")
        i=0
        for joueur in ListeJoueur:
            ListeMise[i]=joueur.mise(self)
            print("Le joueur "+str(i)+ " mise "+str(ListeMise[i]))
            i=i+1
        """distribution des cartes"""
        print("le croupier distribue les cartes:")
        for i in range(len(ListeJoueur)):
            num=self.tireCarte()
            if(num==1):
                ListeAs[i]=ListeAs[i]+1
                ListeJeu[i]=ListeJeu[i]+11
            else:
                ListeJeu[i]=ListeJeu[i]+num
            
        for i in range(len(ListeJoueur)):
            num=self.tireCarte()
            if(num==1):
                ListeAs[i]=ListeAs[i]+1
                ListeJeu[i]=ListeJeu[i]+11
            else:
                ListeJeu[i]=ListeJeu[i]+num
            print("le joueur "+str(i)+" a un total de "+str(ListeJeu[i]))
            if(ListeJeu[i]==21):
                ListeJeu[i]==-1
        num=self.tireCarte()
        Cartecroupier=num
        print("le croupier a la carte "+ str(num))
        i=0
        """les joueurs jouent"""
        print("les joueurs jouent:")
        for joueur in ListeJoueur:
            ListeJeu[i]=joueur.joue(self,ListeJeu[i],ListeAs[i],Cartecroupier)
            if(joueur.double==True):
                joueur.cagnotte=joueur.cagnotte-ListeMise[i]
                ListeMise[i]=ListeMise[i]*2
                joueur.double==False
            print("le joueur "+str(i)+"finit son tour avec un score de "+str(ListeJeu[i]))
            i=i+1
        """le croupier joue"""
        Cartecroupier=croupier.joue(self,Cartecroupier)
        print("le croupier finit avec un score de " + str(Cartecroupier))
        """on distribue les gains"""
        print("resultats")
        
        if(Cartecroupier==0):
            i=0
            for joueur in ListeJoueur:
                comptage=joueur.comptage(self.tas)
                joueur.partieJouer[comptage]=joueur.partieJouer[comptage]+1
                if(ListeJeu[i]>0):
                    joueur.win(ListeMise[i])
                    joueur.compteur[comptage]=joueur.compteur[comptage]+1
                    print("Le joueur "+str(i)+" remporte "+str(ListeMise[i]))
                if(ListeJeu[i]==-1):
                    joueur.win2(ListeMise[i])
                    joueur.compteur[comptage]=joueur.compteur[comptage]+1
                    print("Blackjack pour le joueur " +str(i)+"il remporte"+str(ListeMise[i]*1.5))
                i=i+1
        else:
            i=0
            for joueur in ListeJoueur:
                comptage=joueur.comptage(self.tas)
                joueur.partieJouer[comptage]=joueur.partieJouer[comptage]+1
                if(Cartecroupier==-1):
                    if (ListeJeu[i]==-1):
                        joueur.win(ListeMise[i])
                        joueur.compteur[comptage]=joueur.compteur[comptage]+1
                else:
                    
                    if(ListeJeu[i]>Cartecroupier):
                        joueur.win(ListeMise[i])
                        joueur.compteur[comptage]=joueur.compteur[comptage]+1
                        print("Le joueur "+str(i)+" remporte "+str(ListeMise[i]))
                    if((ListeJeu[i]==-1) and (Cartecroupier!=-1)):
                        print("Blackjack pour le joueur " +str(i)+"il remporte"+str(ListeMise[i]*1.5))
                        joueur.win2(ListeMise[i])
                        joueur.compteur[comptage]=joueur.compteur[comptage]+1
                    if(ListeJeu[i]==Cartecroupier):
                        joueur.egal(ListeMise[i])
                        joueur.partieJouer[comptage]=joueur.partieJouer[comptage]-1
                        print("le joueur "+str(i)+"recupere sa mise initial")
                    i=i+1
        print("les autres joueurs perdent leur mises.")
                
        return 0
 
    def reinit_sabot(self):
        self.sabot=312
        self.tas=[0,24,24,24,24,24,24,24,24,24,96]
        
    def tireCarte(self):
        alea=random.randint(1,self.sabot)
        res=0
        while(alea>0):
            res=res+1
            alea=alea - self.tas[res]
        self.tas[res]=self.tas[res]-1
        self.sabot =self.sabot-1
        if(self.sabot==0):
            self.reinit_sabot()
        return res
            
        
        
class croupier:
    """joueur croupier utiliser dans le blackjack qui peut joueur d'une unique maniere,il tire des cartes jusqu'a depassé 17.
    il est utilisé dans la classe Blackjack et n'a pas d'initialisation.
    la fionction joue permet au croupier de jouer comme decris precedemment"""
    def __init__(self):
        self.age=21
        
    def joue(self,Jeu,carte):
        if(carte==1):
            carte=11
            AS=1
        else :
            AS=0
        
        num=Jeu.tireCarte()
        if(num==1):
            carte=carte+11
            AS=AS+1
       
        for i in range(AS):
            carte=carte+10
        while(AS>=0):
            while(carte<17):
            
                num=Jeu.tireCarte()
                if(num==1):
                    carte=carte+11
                    AS=AS+1
                else:
                    carte=carte+num
            if (carte<=21):
                return carte
            elif(AS==0):
                return 0
            else:
                AS=AS-1
                carte=carte-10
                
class Joueur:
    """ Classe de joueur generique. Necessite une methode joue et mise qui different selon le type de joueur est des fonction win,
    win2 et egal qui met a jour la reserve du jour suivant le resultat d'une partie.Pour l'instant tout les joueur commencent avec 20 euros."""
    def __init__(self,style=None):
        self.cagnotte=100
        self.style=style
        self.double=False
        self.compteur=[0]*30
        self.partieJouer=[0]*30
        pass
    
    def choix_mise(self,Jeu):
        pass
    def mise(self,Jeu):
        nb=self.choix_mise(Jeu)
        self.cagnotte=self.cagnotte-nb
        return nb
    
    def joue(self,Jeu,carte,AS,Cartecroupier):
        pass
    
    def win(self,mise):
        if(self.double==True):
            self.cagnotte=self.cagnotte+4*mise
        else:
            self.cagnotte=self.cagnotte+2*mise
    def win2(self,mise):
        if(self.double==True):
            self.cagnotte=self.cagnotte+2*(1.5*mise+mise)
        else:
            self.cagnotte=self.cagnotte+1.5*mise+mise
    def egal(self,mise):
        self.cagnotte=self.cagnotte+mise
    def getCagnotte(self):
        return self.cagnotte()
    def getStyle(self):
        return self.style
    def comptage(self,tab):
        res=0
        for i in range(2,7):
            res=res+tab[i]
        if(res-tab[10]-tab[1]+15>15):
            return 29
        elif(res-tab[10]-tab[1]+15<15):
            return 0
        else:
            return res-tab[10]-tab[1]+15
    
    
            
            
class JoueurNul(Joueur):
    """joueur de blackjack qui mise toujour 1euro et qui check toujours."""
    def __init__(self,style=None):
        self.cagnotte=100
        self.style="nul"
        self.double=False
        self.compteur=[0]*30
        self.partieJouer=[0]*30
    def choix_mise(self,Jeu):
        return 1
    def joue(self,Jeu,carte,AS,Cartecroupier):
        if(carte==22):
            return 12
        else:
            return carte
    
    
    

In [104]:
class Arbre:
    def __init__(self,tipe,valeur,proba,sabot,fils):
        self.valeur=valeur
        self.proba=proba
        self.sabot=sabot
        self.fils=fils
        self.tipe=tipe
    def supprimer_arbre(self):
        if(self.tipe=="check"):
            del self
        else:
            for i in range(len(self.fils)):
                self.fils[i].supprimer_arbre()


class JoueurCompteur(Joueur):
    """joueur qui comptent les cartes"""
    def __init__(self,style=None):
        self.cagnotte=100
        self.style="Compteur"
        self.double=False
        self.compteur=[0]*30
        self.partieJouer=[0]*30
        #self.train()
        
    def train(self):
        BJ2=Jeu()
        croupier2=croupier()
        self.cagnotte=1000
        prec=1000
        listeJoueur=[self]
        for i in range(1000):
            compte=self.comptage(BJ2.tas)
            BJ2.run(listeJoueur,crouipier2)
            if(self.cagnotte<prec):
                self.compteur[compte]=self.compteur[compte]-1
                self.partieJouer[compte]=self.partieJouer[compte]+1
            if(self.cagnotte>prec):
                self.compteur[compte]=self.compteur[compte]+1
                self.partieJouer[compte]=self.partieJouer[compte]+1
            prec=self.cagnotte
        self.cagnotte=20
            
        
        
    
    
    def choix_mise(self,Jeu):
        compte=self.comptage(Jeu.tas)
        if(self.partieJouer[compte]<20):
            return 1
        else:
            
            return (self.compteur[compte]*self.cagnotte*(1.0))/self.partieJouer[compte]
    
    def joue(self,Jeu,carte,AS,Cartecroupier):
        if(carte==-1):
            return -1
        proba=copi(Jeu.tas[1:])
        arbre=self.creerArbre("init",carte,Jeu.tas,Jeu.sabot,AS)
        tmp=self.calculChemin(arbre)
        print("tmp="+str(tmp))
        while(arbre.tipe!="check"):
            if((arbre.tipe=="init") or (arbre.tipe=="choix")):
                arbre=arbre.fils[0]
            if(arbre.tipe=="double"):
                print("le joueur double")
                self.double=True
                newcarte=Jeu.tireCarte()
                carte=carte+newcarte
                if(carte>21):
                    return 0
                else:
                    arbre=arbre.fils[newcarte-1]
            if(arbre.tipe=="tire"):
                newcarte=Jeu.tireCarte()
                if(newcarte==1):
                    carte=carte+11
                    AS=AS+1
                else:
                    carte=carte+newcarte
                if(carte>21):
                    if(AS==0):
                        return 0
                    else:
                        carte=carte-10
                        AS=AS-10
                
                arbre=arbre.fils[newcarte-1]
        arbre.supprimer_arbre()
        return carte
        
        
        
        
    def creerArbre(self,tipe,carte,proba,sabot,AS):
        if(sabot==0):
            sabot=312
            proba=[0,24,24,24,24,24,24,24,24,24,96]
        if(tipe=="init"):
            A1=self.creerArbre("check",carte,proba,sabot,AS)
            A2=self.creerArbre("tire",carte,proba,sabot,AS)
            A3=self.creerArbre("double",carte,proba,sabot,AS)
            A4=self.creerArbre("split",carte,proba,sabot,AS)
            fils=[A1,A2,A3,A4]
            A=Arbre(tipe,carte,proba,sabot,fils)
            return A
        elif(tipe=="check"):
            A=Arbre(tipe,carte,proba,sabot,[])
            return A
        elif(tipe=="choix"):
            A1=self.creerArbre("check",carte,proba,sabot,AS)
            A2=self.creerArbre("tire",carte,proba,sabot,AS)
            fils=[A1,A2]
            A=Arbre(tipe,carte,proba,sabot,fils)
            return A
        elif(tipe=="tire"):
            if(carte==0):
                A=Arbre("check",carte,proba,sabot,[])
                    
            else:
                fils=[]
                proba[0]=proba[0]-1
                if(carte+11>21):
                    if(carte<21):
                        fils.append(self.creerArbre("choix",carte+1,proba,sabot-1,AS))
                    else:
                        fils.append(self.creerArbre("check",0,proba,sabot-1,AS))
                else:
                    fils.append(self.creerArbre("choix",carte+11,proba,sabot-1,AS+1))
                proba[0]=proba[0]+1
                for i in range(2,11):
                    proba[i-1]=proba[i-1]-1
                    if(carte+i>21):
                        if(AS==0):
                            fils.append(self.creerArbre("check",0,proba,sabot-1,AS))
                        else:
                            fils.append(self.creerArbre("choix",carte+i-10,proba,sabot-1,AS-1))
                    else:
                        fils.append(self.creerArbre("choix",carte+i,proba,sabot-1,AS))
                    proba[i-1]=proba[i-1]+1
            A=Arbre("tire",carte,proba,sabot,fils)
            return A
        elif(tipe=="double"):
            
            fils=[]
            proba[0]=proba[0]-1
            if(carte+11>21):
                fils.append(Arbre("check",carte+1,proba,sabot-1,[]))
            else:
                fils.append(Arbre("check",carte+11,proba,sabot-1,[]))
            proba[0]=proba[0]+1
            for i in range(2,11):
                proba[i-1]=proba[i-1]-1
                if(carte+i>21):
                    if(AS==0):
                        fils.append(Arbre("check",0,proba,sabot-1,[]))
                    else:
                        fils.append(Arbre("check",carte+i-10,proba,sabot-1,[]))
                else:
                    fils.append(Arbre("check",carte+i,proba,sabot-1,[]))
                proba[i-1]=proba[i-1]+1
            A=Arbre("double",carte,proba,sabot,fils)
            return A
            """a ecrire plus tard (tipe=="split")"""
        else:
            A=Arbre("split",carte,proba,sabot,[])
            return A
    
    def calculChemin(self,A):
        if((A.tipe=="check") or (A.tipe=="split")):
            
            return A.valeur
        else:
            if(A.tipe=="tire" or A.tipe=="double"):
                res=0
                for i in range(10):
                    tmp=self.calculChemin(A.fils[i])
                    res=res+A.proba[i]*mt.exp(tmp)
                A.valeur=res*(1.0)/A.sabot
                return mt.log(A.valeur)
            elif(A.tipe=="choix"):
                
                res1=self.calculChemin(A.fils[0])
                res2=self.calculChemin(A.fils[1])
                if (res2>res1):
                    A.fils[0].supprimer_arbre()
                    A.fils=[A.fils[1]]
                    A.valeur=res2
                    return res2
                else:
                    A.fils[1].supprimer_arbre()
                    A.fils=[A.fils[0]]
                    A.valeur=res1
                    return res1
            elif(A.tipe=="init"):
                
                res1=self.calculChemin(A.fils[0])
                res2=self.calculChemin(A.fils[1])
                res3=self.calculChemin(A.fils[2])
                res4=self.calculChemin(A.fils[3])
                if (max(res1,res2,res3,res4)==res3):
                    A.fils[0].supprimer_arbre()
                    A.fils[1].supprimer_arbre()
                    A.fils[3].supprimer_arbre()
                    A.fils=[A.fils[2]]
                    A.valeur=res3
                    return res3
                elif (max(res1,res2,res3,res4)==res1):
                    A.fils[1].supprimer_arbre()
                    A.fils[2].supprimer_arbre()
                    A.fils[3].supprimer_arbre()
                    A.fils=[A.fils[0]]
                    A.valeur=res1
                    return res1
                elif (max(res1,res2,res3,res4)==res2):
                    A.fils[0].supprimer_arbre()
                    A.fils[2].supprimer_arbre()
                    A.fils[3].supprimer_arbre()
                    A.fils=[A.fils[1]]
                    A.valeur=res2
                    return res2

                if (max(res1,res2,res3,res4)==res4):
                    A.fils[0].supprimer_arbre()
                    A.fils[1].supprimer_arbre()
                    A.fils[2].supprimer_arbre()
                    A.fils=[A.fils[3]]
                    A.valeur=res4
                    return res4
                
                
def copi(tab):
    res=[0]*len(tab)
    for i in range(len(tab)):
        res[i]=tab[i]
    return res

In [105]:
croupier=croupier()
j1=JoueurNul()
j2=JoueurNul()
j3=JoueurCompteur()
listeJoueur=[j1,j2,j3]
BJ=Jeu()
for i in range(50):
    
    print(BJ.tas)
    BJ.run(listeJoueur,croupier)

for joueur in listeJoueur:
    res=0
    for i in range(len(joueur.compteur)):
        res=res+joueur.compteur[i]
    print("le joueur "+joueur.style+" a gagné "+str(res)+" parties.")

[0, 24, 24, 24, 24, 24, 24, 24, 24, 24, 96]
NOUVELLE PARTIE:
les joueurs mises:
Le joueur 0 mise 1.0
Le joueur 1 mise 1.0
Le joueur 2 mise 1.0
le croupier distribue les cartes:
le joueur 0 a un total de 15.0
le joueur 1 a un total de 13.0
le joueur 2 a un total de 16.0
le croupier a la carte 10
les joueurs jouent:
le joueur 0finit son tour avec un score de 15.0
le joueur 1finit son tour avec un score de 13.0
tmp=18.92516541990416
le joueur 2finit son tour avec un score de 21.0
le croupier finit avec un score de 20
resultats
Le joueur 2 remporte 1.0
les autres joueurs perdent leur mises.
[0, 24, 24, 23, 24, 22, 23, 23, 24, 24, 91]
NOUVELLE PARTIE:
les joueurs mises:
Le joueur 0 mise 1.0
Le joueur 1 mise 1.0
Le joueur 2 mise 1.0
le croupier distribue les cartes:
le joueur 0 a un total de 20.0
le joueur 1 a un total de 5.0
le joueur 2 a un total de 12.0
le croupier a la carte 7
les joueurs jouent:
le joueur 0finit son tour avec un score de 20.0
le joueur 1finit son tour avec un score de 5

le joueur 1finit son tour avec un score de 16.0
tmp=18.8944616257649
le joueur double
le joueur 2finit son tour avec un score de 0.0
le croupier finit avec un score de 21
resultats
les autres joueurs perdent leur mises.
[0, 12, 13, 15, 5, 14, 10, 7, 7, 13, 45]
NOUVELLE PARTIE:
les joueurs mises:
Le joueur 0 mise 1.0
Le joueur 1 mise 1.0
Le joueur 2 mise 1.0
le croupier distribue les cartes:
le joueur 0 a un total de 18.0
le joueur 1 a un total de 15.0
le joueur 2 a un total de 18.0
le croupier a la carte 2
les joueurs jouent:
le joueur 0finit son tour avec un score de 18.0
le joueur 1finit son tour avec un score de 15.0
tmp=18.900328540496513
le joueur double
le joueur 2finit son tour avec un score de 0.0
le croupier finit avec un score de 0
resultats
Le joueur 0 remporte 1.0
Le joueur 1 remporte 1.0
les autres joueurs perdent leur mises.
[0, 12, 12, 15, 5, 13, 10, 6, 5, 13, 39]
NOUVELLE PARTIE:
les joueurs mises:
Le joueur 0 mise 1.0
Le joueur 1 mise 1.0
Le joueur 2 mise 1.0
le croupi

les joueurs jouent:
le joueur 0finit son tour avec un score de 14.0
le joueur 1finit son tour avec un score de 11.0
tmp=19.061644874582065
le joueur 2finit son tour avec un score de 0.0
le croupier finit avec un score de 21
resultats
les autres joueurs perdent leur mises.
[0, 20, 22, 21, 23, 22, 23, 21, 22, 22, 90]
NOUVELLE PARTIE:
les joueurs mises:
Le joueur 0 mise 1.0
Le joueur 1 mise 1.0
Le joueur 2 mise 34.8
le croupier distribue les cartes:
le joueur 0 a un total de 20.0
le joueur 1 a un total de 14.0
le joueur 2 a un total de 16.0
le croupier a la carte 4
les joueurs jouent:
le joueur 0finit son tour avec un score de 20.0
le joueur 1finit son tour avec un score de 14.0
tmp=18.918833712589773
le joueur 2finit son tour avec un score de 0.0
le croupier finit avec un score de 20
resultats
le joueur 0recupere sa mise initial
les autres joueurs perdent leur mises.
[0, 20, 22, 20, 22, 22, 21, 20, 21, 21, 86]
NOUVELLE PARTIE:
les joueurs mises:
Le joueur 0 mise 1.0
Le joueur 1 mise 1.0


Le joueur 0 mise 1.0
Le joueur 1 mise 1.0
Le joueur 2 mise 1.0
le croupier distribue les cartes:
le joueur 0 a un total de 15.0
le joueur 1 a un total de 9.0
le joueur 2 a un total de 18.0
le croupier a la carte 6
les joueurs jouent:
le joueur 0finit son tour avec un score de 15.0
le joueur 1finit son tour avec un score de 9.0
tmp=18.605211490573485
le joueur double
le joueur 2finit son tour avec un score de 0.0
le croupier finit avec un score de 0
resultats
Le joueur 0 remporte 1.0
Le joueur 1 remporte 1.0
les autres joueurs perdent leur mises.
[0, 6, 6, 7, 5, 6, 8, 7, 4, 5, 32]
NOUVELLE PARTIE:
les joueurs mises:
Le joueur 0 mise 1.0
Le joueur 1 mise 1.0
Le joueur 2 mise 1.0
le croupier distribue les cartes:
le joueur 0 a un total de 11.0
le joueur 1 a un total de 9.0
le joueur 2 a un total de 8.0
le croupier a la carte 5
les joueurs jouent:
le joueur 0finit son tour avec un score de 11.0
le joueur 1finit son tour avec un score de 9.0
tmp=18.31218485127843
le joueur 2finit son tour a

In [29]:
(23.0*13+23*14+22*15+23*16+23*17+21*18+20*19+23*20)*1.0/284

10.309859154929578

In [88]:
mt.exp(20)

485165195.4097903

In [89]:
mt.exp(485165195.4097903)

OverflowError: math range error

In [90]:
mt.log(485165195.4097903)

20.0